In [ ]:
import pandas as pd
from datetime import datetime

# Load datasets
health_df = pd.read_csv("/content/health_monitoring.csv")
safety_df = pd.read_csv("/content/safety_monitoring.csv")
reminder_df = pd.read_csv("/content/daily_reminder.csv")

class ElderlyCareSystem:
    def __init__(self, health_data, safety_data, reminder_data):
        self.health_data = health_data
        self.safety_data = safety_data
        self.reminder_data = reminder_data

    def display_logs(self):
        print("[SYSTEM] Displaying all logs...\n")
        all_ids = set(self.health_data["Device-ID/User-ID"].str.upper()) | \
                  set(self.safety_data["Device-ID/User-ID"].str.upper()) | \
                  set(self.reminder_data["Device-ID/User-ID"].str.upper())

        for user_id in sorted(all_ids):
            self.print_logs_for_user(user_id)

    def search_patient(self, patient_id):
        pid = patient_id.strip().lower()

        all_ids = [*self.health_data["Device-ID/User-ID"].dropna().str.lower(),
                   *self.safety_data["Device-ID/User-ID"].dropna().str.lower(),
                   *self.reminder_data["Device-ID/User-ID"].dropna().str.lower()]

        if pid not in all_ids:
            print(f"[SYSTEM] No records found for {patient_id}")
            return

        # Match correct case-sensitive ID for printing
        matched_id = self._get_actual_id(pid)
        self.print_logs_for_user(matched_id)

    def _get_actual_id(self, lowercase_id):
        all_ids = list(self.health_data["Device-ID/User-ID"].dropna()) + \
                  list(self.safety_data["Device-ID/User-ID"].dropna()) + \
                  list(self.reminder_data["Device-ID/User-ID"].dropna())
        for real_id in all_ids:
            if real_id.lower() == lowercase_id:
                return real_id
        return lowercase_id

    def print_logs_for_user(self, user_id):
        # Health monitoring
        user_health = self.health_data[self.health_data["Device-ID/User-ID"].str.lower() == user_id.lower()]
        for _, row in user_health.iterrows():
            hr = row.get("Heart Rate", "N/A")
            bp = row.get("Blood Pressure", "N/A")
            glucose = row.get("Glucose Level", "N/A")
            print(f"[HealthMonitorAgent] Vitals for {user_id} - HR: {hr}, BP: {bp}, Glucose: {glucose}")
            if row.get("Alert Triggered (Yes/No)") == "Yes":
                print(f"[HealthMonitorAgent] ALERT -> Abnormal vital signs detected for {user_id}!")

        # Safety monitoring
        user_safety = self.safety_data[self.safety_data["Device-ID/User-ID"].str.lower() == user_id.lower()]
        for _, row in user_safety.iterrows():
            activity = row.get("Activity", "N/A")
            print(f"[SafetyMonitorAgent] Activity for {user_id}: {activity}")
            if row.get("Alert Triggered (Yes/No)") == "Yes":
                print(f"[SafetyMonitorAgent] ALERT -> Unusual behavior detected for {user_id}: {activity}")

        # Reminder
        user_reminder = self.reminder_data[self.reminder_data["Device-ID/User-ID"].str.lower() == user_id.lower()]
        for _, row in user_reminder.iterrows():
            reminder_type = row.get("Reminder Type", "N/A")
            if row.get("Reminder Sent (Yes/No)") == "No":
                print(f"[ReminderAgent] Reminder for {user_id}: {reminder_type}")

        # Timestamp
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[TIMESTAMP] {now}")
        print("-" * 70)

    def menu(self):
        while True:
            print("\n===== Elderly Care System Menu =====")
            print("1. See All Logs")
            print("2. Search for Specific Patient/Device ID")
            print("3. Exit")
            choice = input("Enter your choice (1/2/3): ").strip()

            if choice == "1":
                self.display_logs()
            elif choice == "2":
                patient_id = input("Enter Device/Patient ID to search: ").strip()
                self.search_patient(patient_id)
            elif choice == "3":
                print("[SYSTEM] Exiting the program...")
                break
            else:
                print("[SYSTEM] Invalid choice. Please try again.")


print("===== Welcome to the Elderly Care Monitoring System =====")
system = ElderlyCareSystem(health_df, safety_df, reminder_df)
system.menu()



===== Welcome to the Elderly Care Monitoring System =====

===== Elderly Care System Menu =====
1. See All Logs
2. Search for Specific Patient/Device ID
3. Exit
[HealthMonitorAgent] Vitals - HR: 68, BP: 91, Glucose: 77
[SafetyMonitorAgent] Activity: stationary
[SafetyMonitorAgent] ALERT -> Unusual behavior detected: stationary
[ReminderAgent] Reminder: Attend doctor's appointment at 3PM
[HealthMonitorAgent] Vitals - HR: 85, BP: 96, Glucose: 112
[SafetyMonitorAgent] Activity: fall
[SafetyMonitorAgent] ALERT -> Unusual behavior detected: fall
[ReminderAgent] Reminder: Go for a walk
[HealthMonitorAgent] Vitals - HR: 98, BP: 137, Glucose: 128
[SafetyMonitorAgent] Activity: stationary
[SafetyMonitorAgent] ALERT -> Unusual behavior detected: stationary
[ReminderAgent] Reminder: Attend doctor's appointment at 3PM
[HealthMonitorAgent] Vitals - HR: 92, BP: 143, Glucose: 163
[HealthMonitorAgent] ALERT -> Abnormal vital signs detected!
[SafetyMonitorAgent] Activity: stationary
[SafetyMonitorAgent